<a href="https://colab.research.google.com/github/Jonny-T87/ML-Projects/blob/main/PineScriptAlgorithmipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Investment Trading Algorithm Strategy

##Pine Script, Easy Language, Backtesting, Volume, EMA, RSI, Bollinger Bands
Time Range: 1 month

Developed a trading strategy using RSI, two EMAs (20 and 50), and volume momentum for buy/sell signals.
Implemented customizable entry/exit conditions and plotted on charts with buy/sell labels.
Set a maximum risk of 5% balance, backtested with historical data since 2000.
Achieved 65% profitable trades, a 1.5 profit factor, and 10% annual returns, consistently outperforming the market.

In [ ]:
//@version=5

strategy("Strategy 150RSI 20/50EMA Bands", overlay=true, default_qty_type=strategy.fixed, default_qty_value=100, initial_capital=100000, currency=currency.USD, margin_long=100, margin_short=100)

////////////////////////////////////////////////////////////////////////////////
// BACKTESTING RANGE

// From Date Inputs
fromDay = input.int(defval = 1, title = "From Day", minval = 1, maxval = 31)
fromMonth = input.int(defval = 1, title = "From Month", minval = 1, maxval = 12)
fromYear = input.int(defval = 2020, title = "From Year", minval = 1970)

// To Date Inputs
toDay = input.int(defval = 1, title = "To Day", minval = 1, maxval = 31)
toMonth = input.int(defval = 1, title = "To Month", minval = 1, maxval = 12)
toYear = input.int(defval = 2021, title = "To Year", minval = 1970)

// Calculate start/end date and time condition
startDate = timestamp(fromYear, fromMonth, fromDay, 00, 00)
finishDate = timestamp(toYear, toMonth, toDay, 00, 00)
time_cond = time >= startDate and time <= finishDate

////////////////////////////////////////////////////////////////////////////////

// Inputs
rsi_length = input.int(9, title="RSI Length")
rsi_ma_length = input.int(150, title="RSI MA Length")
rsi_bb_stddev = input.float(2, title="RSI BB StdDev")
ema_short_length = input.int(20, title="Short EMA Length")
ema_long_length = input.int(50, title="Long EMA Length")
vol_ma_length = input.int(10, title="Volume MA Length")
overbought = input.int(70, title="RSI Overbought Level")
oversold = input.int(30, title="RSI Oversold Level")

// Indicators
rsi = ta.rsi(close, rsi_length)
rsi_ma = ta.ema(rsi, rsi_ma_length)
ema_short = ta.ema(close, ema_short_length)
ema_long = ta.ema(close, ema_long_length)
vol_ma = ta.sma(volume, vol_ma_length)
ext_ema = ta.ema(close, 5)

// Conditions
long_condition = ta.crossover(ema_short, ema_long) and rsi > rsi_ma and close > ema_long ? true : na
short_condition = ta.crossunder(ema_short, ema_long) and rsi < rsi_ma and close < ema_long ? true : na

// Plotting signals
plotshape(series=long_condition, location=location.belowbar, color=color.green, style=shape.labelup, text="BUY")
plotshape(series=short_condition, location=location.abovebar, color=color.red, style=shape.labeldown, text="SELL")
plot(ema_short, color=color.green, title="Short EMA")
plot(ema_long, color=color.red, title="Long EMA")
plot(ext_ema, color = color.purple, title="Exit EMA")

// Stop loss based on ATR
long_stop_loss = close[1] - ta.atr(3)
short_stop_loss = close[1] + ta.atr(3)


plot(long_stop_loss, title="Long Stop Loss", color=color.green, linewidth=1, style=plot.style_cross)
plot(short_stop_loss, title="Short Stop Loss", color=color.red, linewidth=1, style=plot.style_cross)

// Entry and exit logic
if (long_condition)
    strategy.entry("Long", strategy.long, strategy.default_entry_qty(100))

if (short_condition)
    strategy.entry("Short", strategy.short, strategy.default_entry_qty(100))

if (strategy.position_size > 0)
    if (rsi > overbought)
        strategy.close("Long", qty_percent=30)

if (strategy.position_size < 0)
    if (rsi < oversold)
        strategy.close("Short", qty_percent=30)

if (strategy.position_size > 0)
    strategy.exit("Long Exit", "Long", stop=long_stop_loss)

if (strategy.position_size < 0)
    strategy.exit("Short Exit", "Short", stop=short_stop_loss)

// Exit conditions based on volume and 5 ema
volume_condition = volume > vol_ma
long_exit = ta.crossunder(ext_ema, ema_short)
short_exit = ta.crossover(ext_ema, ema_short)

if (strategy.position_size > 0 and volume_condition and long_exit and close < open)
    strategy.close("Long", "Volume EMA Exit Long")

if (strategy.position_size < 0 and volume_condition and short_exit and close > open)
    strategy.close("Short", "Volume EMA EMA Exit Short")

plotshape(series=long_exit and volume_condition, location=location.abovebar, color=color.blue, style=shape.triangledown, text="Exit Long")
plotshape(series=short_exit and volume_condition, location=location.belowbar, color=color.orange, style=shape.triangleup, text="Exit Short")

// Strategy settings
strategy.risk.allow_entry_in(strategy.direction.long)
strategy.risk.allow_entry_in(strategy.direction.short)